In [3]:
#default_exp azure.filehanling

In [4]:
#hide
from nbdev.showdoc import *
from IPython.display import display, HTML

# ``FileHandling``

> What is Azure Blob? 

Azure Blob storage is Microsoft's object storage solution for the cloud. Blob storage is optimized for storing massive amounts of unstructured data, such as text or binary data.

Blob storage is ideal for:

- Serving images or documents directly to a browser

- Storing files for distributed access

- Streaming video and audio

- Storing data for backup and restore, disaster recovery, and archiving

- Storing data for analysis by an on-premises or Azure-hosted service

In [5]:
#export
from azure.storage.blob import BlobServiceClient
from azure.storage.blob import ContainerClient
from azure.storage.filedatalake import DataLakeServiceClient

import logging
import os
import re
import uuid

logging.basicConfig(level=logging.INFO)
logging.getLogger("azure.core").setLevel(logging.WARNING)
logging.getLogger("urllib3.connectionpool").setLevel(logging.CRITICAL)
logging.getLogger("snowflake.connector").setLevel(logging.WARNING)
logger = logging.getLogger(__name__)

In [6]:
#export


class FileHandling:
    def __init__(self, connection_string, logger=None):
        """
        file handling for all sdsde Azure blob storage. Both upload and download
        have there own clients with in this class so that they can call different
        containers with in the storage account within on init. This is designed,
        but can be removed if we still this to be a waste.
        Args:
        * connection_string (str): azure connection string to blob storage
        * self._logger: logging choices can be overwritten if you want the defaults
        """
        self.connection_string = connection_string
        self.blob_service_client = BlobServiceClient.from_connection_string(connection_string)
        self._logger = logger if logger is not None else logging.getLogger(__name__)

    def upload(self,
               container_name: str,
               file_path: str,
               dest: str = None,
               overwrite: bool = False):
        """
        upload a folder a file to azure blob storage
        Args:
        * container_name (str): azure blob container name
        * file_path (str): file or directory to upload to azure
        * dest (str, optional): destination in azure/ file name in azure. Defaults to ''.
        * overwrite (bool, optional): write over same file names. Defaults to False.
        """
        self.container_name = container_name
        self.container_client = self.blob_service_client.get_container_client(container_name)
        self.create_blob_container(container_name=container_name)
        if (os.path.isdir(file_path)) is not False:
            self.load_dir = True
            self.upload_dir(file_path, dest, overwrite)
        else:
            if dest is not None:
                pass
            else:
                dest = file_path.split('/')[-1]
                self._logger.warning(f'destination to blob is now {dest}')
                self._logger.warning('to delcare new path or file name add dest to upload')
            self.load_dir = False
            self.upload_file(file_path, dest, overwrite)

    def upload_file(self,
                    file_path: str,
                    blob_name: str,
                    overwrite=False):
        """
        simply uploads a file to blob storage
        Args:
        * file_path (str): file to blob
        * blob_name (str): destination in blob
        * overwrite (bool, optional): write over same file names. Defaults to False.
        """
        self._logger.info(f'Uploading {file_path}, to to Azure Storage {blob_name}')
        with open(file_path, "rb") as file:
            try:
                self.container_client.upload_blob(data=file, name=blob_name, overwrite=overwrite)
            except Exception as e:
                self._logger.error(f'"Error Message: {e.error_code.value}"')
        self._logger.info('Azure Upload Complete')

    def upload_dir(self,
                   directory: str,
                   dest: str = None,
                   overwrite: bool = False):
        """
        simply uploads a directory to azure blob note with python
        we can have arguement be used from upload will figure
        that out hopefully
        Args:
        * directory (str): directory that is being moved to blob
        * overwrite (bool, optional): write over same file names. Defaults to False.
        """
        prefix = os.path.basename(directory) + '/'
        # Walk Through Directory
        for root, dirs, files in os.walk(directory):
            for name in files:
                if dest is None:
                    dir_part = os.path.relpath(root, directory)
                    dir_part = '' if dir_part == '.' else dir_part + '/'
                else:
                    dir_part = dest
                file_path = os.path.join(root, name)
                blob_path = prefix + dir_part + name
                self.upload_file(file_path, blob_path, overwrite)

    def create_blob_container(self,
                              container_name: str = str(uuid.uuid4()),
                              unique: bool = False):
        """
        creates/check for container when ``upload`` is called, but
        this function can be used seperately to create
        a new container in isolation within the specific storage account
        Args:
        * container_name (str, optional): creates azure blob container name. Defaults to str(uuid.uuid4()).
        * unique (bool, optional): add a unique tail to container name. Defaults to False.
        """
        regex = re.compile(r'[@_!#$%^&*()<>?/\|}{~:]')
        # Check if container_name is valid
        if (regex.search(container_name) is None):
            self._logger.info(f'{container_name} is a valid')
        else:
            container_name = re.sub(r'[\W_]+', '', container_name)
            self._logger.info(f'container_name changed to {container_name}')
        if unique:
            container_name = container_name + str(uuid.uuid4())
        else:
            container_name = container_name
        # Current Python SDK doesn't support exist
        try:
            # Create the container
            _ = self.blob_service_client.create_container(container_name)
        except Exception as e:
            self._logger.info(f'{e.error_code.value}')

    def download(self,
                 blob_location: str,
                 dest: str,
                 container_name: str,
                 blob_path: str = '',
                 recursive: bool = True,
                 overwrite: bool = False):
        """
        Args:
        * blob_location (str): location in container
        * dest (str): local destination of file
        * container_name (str): azure blob container name.
        * blob_path (str, optional): path in blob helps search. Defaults to ''.
        * recursive (bool, optional): helps with search for file can be false if blob_path is known. Defaults to True.
        * overwrite (bool, optional): write over same file names. Defaults to False.
        """
        if not dest.endswith('/'):
            dest += '/'
        if blob_location.endswith('/'):
            if blob_path == '':
                blobs = self.ls_blob(container_name=container_name, path=blob_path, recursive=recursive)
                blobs = [blobs for blobs in blobs if os.path.basename(os.path.normpath(blob_location)) in blobs]
            else:
                blobs = self.ls_blob(container_name=container_name, path=blob_path, recursive=recursive)
                blobs = [blob_location + blob for blob in blobs]
            for blob in blobs:
                self._logger.info(f'Downloading {blob}')
                self.download_file(container_name, blob, dest, overwrite)
        else:
            self.download_file(container_name, os.path.basename(os.path.normpath(blob_location)), dest, overwrite)
        self._logger.info('Download complete')

    def download_file(self,
                      container_name: str,
                      file: str,
                      file_path: str,
                      overwrite: bool = False):
        """
        Args:
        * container_name (str): azure blob container name.
        * file (str): file to download from blob
        * file_path (str): location to put `file`
        * overwrite (bool, optional): write over same file names. Defaults to False.
        """
        if file_path.endswith('.'):
            file_path += '/'
        blob_dest = file_path + os.path.basename(file) if file_path.endswith('/') else file_path
        self._logger.info(f'{file} to {blob_dest}')
        if not overwrite:
            if os.path.exists(blob_dest):
                self._logger.warning('file path already exist change overwrite to ``True`` if you want to overwrite file')
                return
        os.makedirs(os.path.dirname(blob_dest), exist_ok=True)
        download_client = self.blob_service_client.get_container_client(container_name)
        downloader = download_client.get_blob_client(blob=file)
        with open(blob_dest, 'wb') as file:
            data = downloader.download_blob()
            file.write(data.readall())

    def ls_blob(self,
                container_name: str,
                path: str,
                recursive: bool = False):
        """
        Args:
        * container_name (str): azure blob container name.
        * path (str): blob path to look at
        * recursive (bool, optional): recurisve look. Defaults to False.

        Returns:
        * list: file list
        """
        if not path == '' and not path.endswith('/'):
            path += '/'
        download_client = self.blob_service_client.get_container_client(container_name)
        blob_looker = download_client.list_blobs(name_starts_with=path)
        files = []
        for blob in blob_looker:
            relative_path = os.path.relpath(blob.name, path)
            if recursive or not '/' in relative_path:  # NOQA:
                files.append(relative_path)
        return files

    def rm_files(self,
                 container_name: str,
                 delete_path: str = '',
                 recursive: bool = False):
        """
        removes files from storage account
        Args:
        * container_name (str): azure blob container name.
        * delete_path (str, optional): what to delete file or directory. Defaults to ''.
        * recursive (bool, optional): recursive delete. Defaults to False.
        """
        if not delete_path == '' and not delete_path.endswith('/'):
            delete_path, delete_file = delete_path.rsplit('/', 1)
            delete_path += '/'
            blobs = self.ls_blob(container_name, delete_path, recursive)
            if delete_file in blobs:
                blobs = [delete_file]
        else:
            blobs = self.ls_blob(container_name, delete_path, recursive)
        if not blobs:
            self._logger.warning('location in blob is empty')
            return
        blobs = [delete_path + blob for blob in blobs]
        self._logger.info(f'files to be removed {blobs}')
        delete_client = self.blob_service_client.get_container_client(container_name)
        if len(blobs) > 1:
            for x in blobs:
                delete_client.delete_blobs(*x)
        else:
            delete_client.delete_blob(*blobs)

    def rm_folder(self, container_name: str, folder_path: str):
        """Doesn't work need to build"""
        container_client = ContainerClient.from_connection_string(conn_str=self.connection_string,
                                                                  container_name=container_name)
        container_client.delete_blob(blob=folder_path)

    def rm_container(self,
                     container_name: str):
        "remove container from storage account"
        self.blob_service_client.delete_container(container_name)

    def ls_containers(self,
                      name_starts_with: str = None):
        "show containers in storage account"
        container_names = self.blob_service_client.list_containers(name_starts_with=name_starts_with)
        for names in container_names:
            print(names['name'])

In [7]:
#export


def unlink_files(files: list, file_path: str = './'):
    """
    clean up tool for files that shouldn't be there

    Args:
    * files (list): can be a list of just one file to remove
    * file_path (str, optional): location of the file. Defaults to './'.
    """
    file_list = files
    for x in file_list:
        os.unlink(os.path.join(file_path, x))

In [8]:
show_doc(FileHandling)

<h2 id="FileHandling" class="doc_header"><code>class</code> <code>FileHandling</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>FileHandling</code>(**`connection_string`**, **`logger`**=*`None`*)



# How To Use ``FileHandling``

In [9]:
#hide
connect_str = os.environ['connection_str']
fh = FileHandling(connect_str)

# Upload

This Section will show how a user can send files up to azure for storage

In [10]:
show_doc(FileHandling.upload)

<h4 id="FileHandling.upload" class="doc_header"><code>FileHandling.upload</code><a href="__main__.py#L19" class="source_link" style="float:right">[source]</a></h4>

> <code>FileHandling.upload</code>(**`container_name`**:`str`, **`file_path`**:`str`, **`dest`**:`str`=*`None`*, **`overwrite`**:`bool`=*`False`*)

upload a folder a file to azure blob storage
Args:
* container_name (str): azure blob container name
* file_path (str): file or directory to upload to azure
* dest (str, optional): destination in azure/ file name in azure. Defaults to ''.
* overwrite (bool, optional): write over same file names. Defaults to False.

## Send a Directory To Azure

When the ``file_path`` is specified as a folder the entire folder will be sent to azure blob.

In [11]:
# Upload dir 
fh.upload(container_name='sdsdetestazurenotebook',
          file_path='testing/',
          dest='testing/',
          overwrite=True)

INFO:__main__:sdsdetestazurenotebook is a valid
INFO:__main__:Uploading testing/clean_multi.sql, to to Azure Storage /testing/clean_multi.sql
INFO:__main__:Azure Upload Complete
INFO:__main__:Uploading testing/multi.sql, to to Azure Storage /testing/multi.sql
INFO:__main__:Azure Upload Complete
INFO:__main__:Uploading testing/sdsde_test.yaml, to to Azure Storage /testing/sdsde_test.yaml
INFO:__main__:Azure Upload Complete
INFO:__main__:Uploading testing/sftp.yaml, to to Azure Storage /testing/sftp.yaml
INFO:__main__:Azure Upload Complete
INFO:__main__:Uploading testing/sftp_only.yaml, to to Azure Storage /testing/sftp_only.yaml
INFO:__main__:Azure Upload Complete
INFO:__main__:Uploading testing/sftp_query.sql, to to Azure Storage /testing/sftp_query.sql
INFO:__main__:Azure Upload Complete
INFO:__main__:Uploading testing/snowflake.yaml, to to Azure Storage /testing/snowflake.yaml
INFO:__main__:Azure Upload Complete
INFO:__main__:Uploading testing/test.txt, to to Azure Storage /testing/t

In [12]:
# Check if files arrived
assert fh.ls_blob(container_name='sdsdetestazurenotebook', path='testing/', recursive=True) == ['clean_multi.sql',
 'multi.sql',
 'sdsde_test.yaml',
 'sftp.yaml',
 'sftp_only.yaml',
 'sftp_query.sql',
 'snowflake.yaml',
 'test.txt',
 'test_df.csv',
 'test_yaml.sql',
 'view_test.sql'], 'Test folder should be in azure'

## Send A File To Azure Blob

Having just a file be sent up to azure is as simple as giving it a file path and it will take the file name and add it to the root directory of the ``container_name``.

If you want to change the name or sent this file to a certain location with in the container_name ``dest`` option allows you to specify the name of the file as well as the location that it will be dropped off at in azure.

In [13]:
# Upload file using actual file name in root 
fh.upload(container_name='sdsdetestazurenotebook',
          file_path='testing/test.txt',
          overwrite=True)

assert 'test.txt' in fh.ls_blob(container_name='sdsdetestazurenotebook', path='', recursive=False), 'test.txt should be in the root'

INFO:__main__:sdsdetestazurenotebook is a valid
INFO:__main__:ContainerAlreadyExists
INFO:__main__:Uploading testing/test.txt, to to Azure Storage test.txt
INFO:__main__:Azure Upload Complete


## Send a File to Azure Blob and Rename File on the Way

In [14]:
# Upload file with changed name
fh.upload(container_name='sdsdetestazurenotebook',
          file_path='testing/test.txt',
          dest='trainwreck.txt',
          overwrite=True)

assert 'trainwreck.txt' in fh.ls_blob(container_name='sdsdetestazurenotebook', path='', recursive=False), 'test.txt & trainweck should be there'

INFO:__main__:sdsdetestazurenotebook is a valid
INFO:__main__:ContainerAlreadyExists
INFO:__main__:Uploading testing/test.txt, to to Azure Storage trainwreck.txt
INFO:__main__:Azure Upload Complete


# Download Examples

This section show how you can pull files down from azure blob as long as ``FileHandling`` is instantiated 

In [15]:
show_doc(FileHandling.download)

<h4 id="FileHandling.download" class="doc_header"><code>FileHandling.download</code><a href="__main__.py#L121" class="source_link" style="float:right">[source]</a></h4>

> <code>FileHandling.download</code>(**`blob_location`**:`str`, **`dest`**:`str`, **`container_name`**:`str`, **`blob_path`**:`str`=*`''`*, **`recursive`**:`bool`=*`True`*, **`overwrite`**:`bool`=*`False`*)

Args:
* blob_location (str): location in container
* dest (str): local destination of file
* container_name (str): azure blob container name.
* blob_path (str, optional): path in blob helps search. Defaults to ''.
* recursive (bool, optional): helps with search for file can be false if blob_path is known. Defaults to True.
* overwrite (bool, optional): write over same file names. Defaults to False.

## Download a Directory From Blob

In [16]:
# download calls download_file when downloading directory of files
fh.download(blob_location = 'testing/',
            dest = 'testing2/',
            container_name='sdsdetestazurenotebook',
            blob_path = '',
            recursive = True,
            overwrite = True)
assert os.path.exists('testing/multi.sql') == True, 'File should be located here'
unlink_files(['clean_multi.sql', 'multi.sql', 'test.txt', 'view_test.sql', 'testing'], './testing2/')
assert os.path.exists('testing2/multi.sql') == False, 'File should not be there'
! rm -rf testing2/

INFO:__main__:Downloading testing
INFO:__main__:testing to testing2/testing
INFO:__main__:Downloading testing/clean_multi.sql
INFO:__main__:testing/clean_multi.sql to testing2/clean_multi.sql
INFO:__main__:Downloading testing/multi.sql
INFO:__main__:testing/multi.sql to testing2/multi.sql
INFO:__main__:Downloading testing/sdsde_test.yaml
INFO:__main__:testing/sdsde_test.yaml to testing2/sdsde_test.yaml
INFO:__main__:Downloading testing/sftp.yaml
INFO:__main__:testing/sftp.yaml to testing2/sftp.yaml
INFO:__main__:Downloading testing/sftp_only.yaml
INFO:__main__:testing/sftp_only.yaml to testing2/sftp_only.yaml
INFO:__main__:Downloading testing/sftp_query.sql
INFO:__main__:testing/sftp_query.sql to testing2/sftp_query.sql
INFO:__main__:Downloading testing/snowflake.yaml
INFO:__main__:testing/snowflake.yaml to testing2/snowflake.yaml
INFO:__main__:Downloading testing/test.txt
INFO:__main__:testing/test.txt to testing2/test.txt
INFO:__main__:Downloading testing/test_df.csv
INFO:__main__:te

## Download a Specific File From Blob

In [17]:
show_doc(FileHandling.download_file)

<h4 id="FileHandling.download_file" class="doc_header"><code>FileHandling.download_file</code><a href="__main__.py#L153" class="source_link" style="float:right">[source]</a></h4>

> <code>FileHandling.download_file</code>(**`container_name`**:`str`, **`file`**:`str`, **`file_path`**:`str`, **`overwrite`**:`bool`=*`False`*)

Args:
* container_name (str): azure blob container name.
* file (str): file to download from blob
* file_path (str): location to put `file`
* overwrite (bool, optional): write over same file names. Defaults to False.

In [18]:
fh.download_file(file='testing/multi.sql',
                 file_path = 'testing/',
                 container_name='sdsdetestazurenotebook',
                 overwrite = True)

INFO:__main__:testing/multi.sql to testing/multi.sql


# Delete

In [19]:
show_doc(FileHandling.rm_files)

<h4 id="FileHandling.rm_files" class="doc_header"><code>FileHandling.rm_files</code><a href="__main__.py#L204" class="source_link" style="float:right">[source]</a></h4>

> <code>FileHandling.rm_files</code>(**`container_name`**:`str`, **`delete_path`**:`str`=*`''`*, **`recursive`**:`bool`=*`False`*)

removes files from storage account
Args:
* container_name (str): azure blob container name.
* delete_path (str, optional): what to delete file or directory. Defaults to ''.
* recursive (bool, optional): recursive delete. Defaults to False.

In [20]:
fh.rm_files(container_name='sdsdetestazurenotebook', delete_path='testing/view_test.sql')
assert 'view_test.txt' not in fh.ls_blob(container_name='sdsdetestazurenotebook', path='testing/', recursive=False)

INFO:__main__:files to be removed ['testing/view_test.sql']


## Delete a Specific File inside Container

In [21]:
# remove files from blob
# Note: remember when there is only one file in the folder the folder will be deleted
fh.rm_files(container_name='sdsdetestazurenotebook', 
            delete_path='testing/test.txt')
assert 'test.txt' not in fh.ls_blob(container_name='sdsdetestazurenotebook', path='testing', recursive=True), 'only test.txt should be removed'

INFO:__main__:files to be removed ['testing/test.txt']


In [22]:
show_doc(FileHandling.rm_container)

<h4 id="FileHandling.rm_container" class="doc_header"><code>FileHandling.rm_container</code><a href="__main__.py#L241" class="source_link" style="float:right">[source]</a></h4>

> <code>FileHandling.rm_container</code>(**`container_name`**:`str`)

remove container from storage account

## Remove Container Fully

In [23]:
# remove container
fh.rm_container(container_name='sdsdetestazurenotebook')

## View

In [24]:
show_doc(FileHandling.ls_blob)

<h4 id="FileHandling.ls_blob" class="doc_header"><code>FileHandling.ls_blob</code><a href="__main__.py#L180" class="source_link" style="float:right">[source]</a></h4>

> <code>FileHandling.ls_blob</code>(**`container_name`**:`str`, **`path`**:`str`, **`recursive`**:`bool`=*`False`*)

Args:
* container_name (str): azure blob container name.
* path (str): blob path to look at
* recursive (bool, optional): recurisve look. Defaults to False.

Returns:
* list: file list

In [25]:
show_doc(FileHandling.ls_containers)

<h4 id="FileHandling.ls_containers" class="doc_header"><code>FileHandling.ls_containers</code><a href="__main__.py#L246" class="source_link" style="float:right">[source]</a></h4>

> <code>FileHandling.ls_containers</code>(**`name_starts_with`**:`str`=*`None`*)

show containers in storage account

In [26]:
fh.ls_containers(name_starts_with='sdsde_test')

## Create Container

In [27]:
show_doc(FileHandling.create_blob_container)

<h4 id="FileHandling.create_blob_container" class="doc_header"><code>FileHandling.create_blob_container</code><a href="__main__.py#L92" class="source_link" style="float:right">[source]</a></h4>

> <code>FileHandling.create_blob_container</code>(**`container_name`**:`str`=*`'0910a90c-50ea-4bd0-ba18-71126b803cd4'`*, **`unique`**:`bool`=*`False`*)

creates/check for container when ``upload`` is called, but
this function can be used seperately to create
a new container in isolation within the specific storage account
Args:
* container_name (str, optional): creates azure blob container name. Defaults to str(uuid.uuid4()).
* unique (bool, optional): add a unique tail to container name. Defaults to False.

In [28]:
fh.create_blob_container(container_name='sdsdetest')
assert fh.ls_blob(container_name='sdsdetest', path='', recursive=True) == [], "empty container should be created"

INFO:__main__:sdsdetest is a valid


In [29]:
# remove container
fh.rm_container(container_name='sdsdetest')

# Data Lake Functions

## `rename_adls_directory`

In [30]:
#export
def rename_adls_directory(old_dir, new_dir, storage_account_name, storage_account_key, container):
    """renames a directory in azure data lake

    Args:
    * old_dir (str): name of old directory
    * new_dir (str): new directory name
    * storage_account_name (str): adls storage account name
    * storage_account_key (str): adls storage account secret key (not connection string)
    * container (str): adls container
    """

    # first connect
    try:
        service_client = DataLakeServiceClient(account_url="{}://{}.dfs.core.windows.net".format(
            "https", storage_account_name), credential=storage_account_key)
    except Exception as e:
        print(e)
    logger.info('Connected to data lake')

    # now rename
    try:
        file_system_client = service_client.get_file_system_client(file_system=container)
        directory_client = file_system_client.get_directory_client(old_dir)
        logger.info('new_dir - ' + directory_client.file_system_name + '/' + new_dir)
        logger.info('old_dir - ' + directory_client.file_system_name + '/' + old_dir)
        directory_client.rename_directory(new_name=directory_client.file_system_name + '/' + new_dir)
    except Exception as e:
        print(e)

In [31]:
show_doc(rename_adls_directory)

<h4 id="rename_adls_directory" class="doc_header"><code>rename_adls_directory</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>rename_adls_directory</code>(**`old_dir`**, **`new_dir`**, **`storage_account_name`**, **`storage_account_key`**, **`container`**)

renames a directory in azure data lake

Args:
* old_dir (str): name of old directory
* new_dir (str): new directory name
* storage_account_name (str): adls storage account name
* storage_account_key (str): adls storage account secret key (not connection string)
* container (str): adls container

In [32]:
old_path = 'old-dir/'
new_path = 'new-dir/'

fh = FileHandling(os.environ['DATALAKE_CONN_STR_SECRET'])
fh.create_blob_container(container_name='sdsdedirectorytest')
fh.upload(container_name='sdsdedirectorytest',
          file_path='testing/test.txt',
          dest=old_path+'test.txt',
          overwrite=True)

rename_adls_directory(old_dir=old_path,
                      new_dir=new_path,
                      storage_account_name=os.environ['azure_account'], 
                      storage_account_key=os.environ['DATALAKE_SECRET'], 
                      container='sdsdedirectorytest')

fh.rm_container(container_name='sdsdedirectorytest')

INFO:__main__:sdsdedirectorytest is a valid
INFO:__main__:ContainerAlreadyExists
INFO:__main__:sdsdedirectorytest is a valid
INFO:__main__:ContainerAlreadyExists
INFO:__main__:Uploading testing/test.txt, to to Azure Storage old-dir/test.txt
INFO:__main__:Azure Upload Complete
INFO:__main__:Connected to data lake
INFO:__main__:new_dir - sdsdedirectorytest/new-dir/
INFO:__main__:old_dir - sdsdedirectorytest/old-dir/


# Create

In [33]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 01_azure.ipynb.
Converted 02_utils_dataframes.ipynb.
Converted 02_utils_parseyaml.ipynb.
Converted 02_utils_stfp.ipynb.
Converted 02_utils_traininghelpers.ipynb.
Converted 02_utils_traininghelpers_fastai.ipynb.
Converted 03_dstools_preparedata.ipynb.
Converted 04_snowflake_copyinto.ipynb.
Converted 04_snowflake_copyinto2.ipynb.
Converted 04_snowflake_query.ipynb.
Converted 05_azure_wrappers.ipynb.
Converted 06_modeling_inference.ipynb.
Converted 06_modeling_inference_fastai.ipynb.
Converted 06_modeling_premodel.ipynb.
Converted 06_modeling_preprocessing.ipynb.
Converted 06_modeling_preprocessing_fastai.ipynb.
Converted 06_modeling_training.ipynb.
Converted 06_modeling_training_fastai.ipynb.
Converted 07_Binary_Classification_Fastai_Example_Notebook.ipynb.
Converted index.ipynb.
